# Visualize suspected RBCs

#### This notebook labels each cell within this patient's images by it's Phenograph cluster number generated in Phenograph-RGB_features.ipynb. Suspected RBCs are outlined in red to visually confirm.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tifffile
import cv2
from PIL import Image
from PIL import ImageDraw
from skimage import data, color, io, img_as_float
from os import listdir
from os.path import isfile, join
import skimage.io
import skimage.util
from skimage import data, segmentation, util, measure
from skimage.segmentation import expand_labels

In [2]:
# create dictionary of directories for each patient
img_dirs = {'P51':'/data/Zhaolab/1_AMLCosMx/Final_scripts/1_Normalization/0_NormalizedImg/P51_R1158_S1_Normalized/', 
            'P52':'/data/Zhaolab/1_AMLCosMx/Final_scripts/1_Normalization/0_NormalizedImg/P52_R1149_S1_Normalized/', 
            'P53':'/data/Zhaolab/1_AMLCosMx/Final_scripts/1_Normalization/0_NormalizedImg/P53_R1149_S2_Normalized/', 
            'P56':'/data/Zhaolab/1_AMLCosMx/Final_scripts/1_Normalization/0_NormalizedImg/P56_R1158_S2_Normalized/', 
            'P57':'/data/Zhaolab/1_AMLCosMx/Final_scripts/1_Normalization/0_NormalizedImg/P57_R1158_S3_Normalized/', 
            'P58':'/data/Zhaolab/1_AMLCosMx/Final_scripts/1_Normalization/0_NormalizedImg/P58_R1149_S3_Normalized/'}

In [3]:
patient = 'P52'
tp = 'A'

In [23]:
# create dictionary of directories for each patient
mask_dir = '/data/Zhaolab/1_AMLCosMx/Final_scripts/2_Segmentation/3_NucMemMerging/' + patient + '_hybrid/labels_predicted_2_15_24/'

## All but FOV07

In [27]:
# load in celltype predictions for all cells
morph_predicted = pd.read_csv(patient + tp + '_RGB_clusters_exFOV07.csv')
morph_predicted

,Unnamed: 0,FOV,cell_ID,Timepoint,PhenoGraph_clusters
0,P52_FOV01_cell_1,1,1,A,1
1,P52_FOV01_cell_2,1,2,A,2
2,P52_FOV01_cell_3,1,3,A,6
3,P52_FOV01_cell_4,1,4,A,1
4,P52_FOV01_cell_5,1,5,A,1
...,...,...,...,...,...
34442,P52_FOV06_cell_13824,6,13824,A,12
34443,P52_FOV06_cell_13825,6,13825,A,9
34444,P52_FOV06_cell_13826,6,13826,A,2
34445,P52_FOV06_cell_13827,6,13827,A,12


In [28]:
morph_predicted = morph_predicted.drop(columns=["Unnamed: 0"])
morph_predicted

,FOV,cell_ID,Timepoint,PhenoGraph_clusters
0,1,1,A,1
1,1,2,A,2
2,1,3,A,6
3,1,4,A,1
4,1,5,A,1
...,...,...,...,...
34442,6,13824,A,12
34443,6,13825,A,9
34444,6,13826,A,2
34445,6,13827,A,12


In [29]:
# add col to indicate if cell is predicted RBC
morph_predicted['RBC'] = 0
morph_predicted['RBC'][morph_predicted['PhenoGraph_clusters'].isin([12])] = 1
morph_predicted

/tmp/ipykernel_1265791/2151582929.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  morph_predicted['RBC'][morph_predicted['PhenoGraph_clusters'].isin([12])] = 1


,FOV,cell_ID,Timepoint,PhenoGraph_clusters,RBC
0,1,1,A,1,0
1,1,2,A,2,0
2,1,3,A,6,0
3,1,4,A,1,0
4,1,5,A,1,0
...,...,...,...,...,...
34442,6,13824,A,12,1
34443,6,13825,A,9,0
34444,6,13826,A,2,0
34445,6,13827,A,12,1


In [30]:
morph_predicted[morph_predicted['RBC'] == 1]

,FOV,cell_ID,Timepoint,PhenoGraph_clusters,RBC
3018,1,3119,A,12,1
3019,1,3120,A,12,1
3020,1,3124,A,12,1
3023,1,3131,A,12,1
3024,1,3134,A,12,1
...,...,...,...,...,...
34408,6,13738,A,12,1
34413,6,13748,A,12,1
34433,6,13812,A,12,1
34442,6,13824,A,12,1


In [31]:
# save list of all patient's RBCs
pos_to_save = morph_predicted[morph_predicted['RBC'] == 1][['FOV', 'cell_ID']]
pos_to_save['Patient'] = patient

### Visualize all cells with cluster names, highlight likely RBCs

In [32]:
# create list of fovs
fovs = list(set(morph_predicted['FOV'].tolist()))
fovs.sort()
    
for fov in fovs:
    
    # subset FOV        
    one_fov = morph_predicted[morph_predicted['FOV'] == fov]
    
    # Load img
    location = img_dirs[patient]
    dirlist = listdir(location)
    dirlist.sort()
    for item in dirlist:
        if 'F0' + str(fov).zfill(2) in item:
            img = cv2.imread(location + item)
            print(item)
            break
            
    # brighten image
    # define the alpha and beta
    alpha = 1.5 # Contrast control
    beta = 20 # Brightness control

    # call convertScaleAbs function
    img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)

    # Load masks
    location = mask_dir
    dirlist = listdir(location)
    dirlist.sort()
    for item in dirlist:
        if 'F0' + str(fov).zfill(2) in item:
            labels = tifffile.imread(location + item)
            print(item)
            break
        elif 'FOV' + str(fov).zfill(2) in item:
            labels = tifffile.imread(location + item)
            print(item)
            break

    # get all mask values
    fov_cells = np.unique(labels)
    
    # Load centroids
    coords_dir = mask_dir + 'cell_centroids/'
    dirlist = listdir(coords_dir)
    dirlist.sort()
    for item in dirlist:
        if 'F0' + str(fov).zfill(2) in item:
            centroids = pd.read_csv(coords_dir + item, index_col=0)
            break
        elif 'FOV' + str(fov).zfill(2) in item:
            centroids = pd.read_csv(coords_dir + item, index_col=0)
            break
            
    # merge prediction and centroid tables
    one_fov['cell_ID'] = one_fov['cell_ID'].astype(int)
    one_fov = one_fov.rename(columns={"cell_ID": "label"})
                
    combo = pd.merge(one_fov,centroids,left_on='label',right_on='label',how='left')
    
    # visualize all segmented cells
    boundaries = segmentation.find_boundaries(labels, connectivity=1, mode='inner', background=0)
    boundaries = boundaries.astype(int)
    img[boundaries>0] = [255,255,0]
    
    # create list of RBCs
    pos_cells = combo[combo['RBC'] == 1]['label'].tolist()
    pos_cells = [int(i) for i in pos_cells]
    # set all labels of other clusters to zero
    labels_edited = labels.copy()
    for cell in fov_cells:
        if cell in pos_cells:
            pass
        else:
            labels_edited[labels_edited == cell] = 0

    # add colored outline
    boundaries = segmentation.find_boundaries(labels_edited, connectivity=1, mode='thick', background=0)
    boundaries = boundaries.astype(int)
    img[boundaries>0] = [0,0,255]

    # get unique phenograph clusters
    clusters = list(set(combo['PhenoGraph_clusters'].tolist()))
    
    for cluster in clusters:

        # create list of this cluster's cells
        pos_cells_df = combo[combo['PhenoGraph_clusters'] == cluster]

        for k in range(pos_cells_df.shape[0]):
            img = cv2.putText(img, str(cluster), (int(pos_cells_df['x'].iloc[k]), int(pos_cells_df['y'].iloc[k])), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [255,255,255], 2, cv2.LINE_AA)
            
    # save image
    cv2.imwrite('RGB_clusters_overlaid/' + patient + '_FOV' + str(fov).zfill(2) + '_RGB_cluster.png', img)

20220215_094003_S1_C902_P99_N99_F001_normalized.png
20220215_094003_S1_C902_P99_N99_F001_normalized_cp_masks.tif


/tmp/ipykernel_1265791/3996469836.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov['cell_ID'] = one_fov['cell_ID'].astype(int)


20220215_094003_S1_C902_P99_N99_F002_normalized.png
20220215_094003_S1_C902_P99_N99_F002_normalized_cp_masks.tif


/tmp/ipykernel_1265791/3996469836.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov['cell_ID'] = one_fov['cell_ID'].astype(int)


20220215_094003_S1_C902_P99_N99_F003_normalized.png
20220215_094003_S1_C902_P99_N99_F003_normalized_cp_masks.tif


...

/tmp/ipykernel_1265791/3996469836.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov['cell_ID'] = one_fov['cell_ID'].astype(int)


20220215_094003_S1_C902_P99_N99_F005_normalized.png
20220215_094003_S1_C902_P99_N99_F005_normalized_cp_masks.tif


/tmp/ipykernel_1265791/3996469836.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov['cell_ID'] = one_fov['cell_ID'].astype(int)


20220215_094003_S1_C902_P99_N99_F006_normalized.png
20220215_094003_S1_C902_P99_N99_F006_normalized_cp_masks.tif


/tmp/ipykernel_1265791/3996469836.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_fov['cell_ID'] = one_fov['cell_ID'].astype(int)


In [33]:
# save IDs of RBCs
pos_to_save.to_csv(patient + tp + '_RBCs_exFOV07.csv', index=False)

## FOV07 only

In [40]:
# load in celltype predictions for all cells
morph_predicted = pd.read_csv(patient + tp + '_RGB_clusters_FOV07.csv')
morph_predicted

,Unnamed: 0,FOV,cell_ID,Timepoint,PhenoGraph_clusters
0,P52_FOV07_cell_1,7,1,A,4
1,P52_FOV07_cell_2,7,2,A,0
2,P52_FOV07_cell_3,7,3,A,4
3,P52_FOV07_cell_4,7,4,A,4
4,P52_FOV07_cell_5,7,5,A,0
...,...,...,...,...,...
7463,P52_FOV07_cell_8809,7,8809,A,0
7464,P52_FOV07_cell_8810,7,8810,A,8
7465,P52_FOV07_cell_8811,7,8811,A,3
7466,P52_FOV07_cell_8812,7,8812,A,3


In [41]:
morph_predicted = morph_predicted.drop(columns=["Unnamed: 0"])
morph_predicted

,FOV,cell_ID,Timepoint,PhenoGraph_clusters
0,7,1,A,4
1,7,2,A,0
2,7,3,A,4
3,7,4,A,4
4,7,5,A,0
...,...,...,...,...
7463,7,8809,A,0
7464,7,8810,A,8
7465,7,8811,A,3
7466,7,8812,A,3


In [42]:
# add col to indicate if cell is predicted RBC
morph_predicted['RBC'] = 0
morph_predicted['RBC'][morph_predicted['PhenoGraph_clusters'].isin([1,2,3,5,6,8,9])] = 1
morph_predicted

/tmp/ipykernel_1265791/1916883790.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  morph_predicted['RBC'][morph_predicted['PhenoGraph_clusters'].isin([1,2,3,5,6,8,9])] = 1


,FOV,cell_ID,Timepoint,PhenoGraph_clusters,RBC
0,7,1,A,4,0
1,7,2,A,0,0
2,7,3,A,4,0
3,7,4,A,4,0
4,7,5,A,0,0
...,...,...,...,...,...
7463,7,8809,A,0,0
7464,7,8810,A,8,1
7465,7,8811,A,3,1
7466,7,8812,A,3,1


In [43]:
morph_predicted[morph_predicted['RBC'] == 1]

,FOV,cell_ID,Timepoint,PhenoGraph_clusters,RBC
10,7,11,A,6,1
64,7,65,A,6,1
162,7,163,A,1,1
194,7,195,A,6,1
238,7,239,A,1,1
...,...,...,...,...,...
7462,7,8808,A,3,1
7464,7,8810,A,8,1
7465,7,8811,A,3,1
7466,7,8812,A,3,1


In [44]:
# save list of all patient's RBCs
pos_to_save = morph_predicted[morph_predicted['RBC'] == 1][['FOV', 'cell_ID']]
pos_to_save['Patient'] = patient

### Visualize all cells with cluster names, highlight likely RBCs

In [45]:
# create list of fovs
fovs = list(set(morph_predicted['FOV'].tolist()))
fovs.sort()
    
for fov in fovs:
    
    # subset FOV        
    one_fov = morph_predicted[morph_predicted['FOV'] == fov]
    
    # Load img
    location = img_dirs[patient]
    dirlist = listdir(location)
    dirlist.sort()
    for item in dirlist:
        if 'F0' + str(fov).zfill(2) in item:
            img = cv2.imread(location + item)
            print(item)
            break
            
    # brighten image
    # define the alpha and beta
    alpha = 1.5 # Contrast control
    beta = 20 # Brightness control

    # call convertScaleAbs function
    img = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)

    # Load masks
    location = mask_dir
    dirlist = listdir(location)
    dirlist.sort()
    for item in dirlist:
        if 'F0' + str(fov).zfill(2) in item:
            labels = tifffile.imread(location + item)
            print(item)
            break
        elif 'FOV' + str(fov).zfill(2) in item:
            labels = tifffile.imread(location + item)
            print(item)
            break

    # get all mask values
    fov_cells = np.unique(labels)
    
    # Load centroids
    coords_dir = mask_dir + 'cell_centroids/'
    dirlist = listdir(coords_dir)
    dirlist.sort()
    for item in dirlist:
        if 'F0' + str(fov).zfill(2) in item:
            centroids = pd.read_csv(coords_dir + item, index_col=0)
            break
        elif 'FOV' + str(fov).zfill(2) in item:
            centroids = pd.read_csv(coords_dir + item, index_col=0)
            break
            
    # merge prediction and centroid tables
    one_fov['cell_ID'] = one_fov['cell_ID'].astype(int)
    one_fov = one_fov.rename(columns={"cell_ID": "label"})
                
    combo = pd.merge(one_fov,centroids,left_on='label',right_on='label',how='left')
    
    # visualize all segmented cells
    boundaries = segmentation.find_boundaries(labels, connectivity=1, mode='inner', background=0)
    boundaries = boundaries.astype(int)
    img[boundaries>0] = [255,255,0]
    
    # create list of RBCs
    pos_cells = combo[combo['RBC'] == 1]['label'].tolist()
    pos_cells = [int(i) for i in pos_cells]
    # set all labels of other clusters to zero
    labels_edited = labels.copy()
    for cell in fov_cells:
        if cell in pos_cells:
            pass
        else:
            labels_edited[labels_edited == cell] = 0

    # add colored outline
    boundaries = segmentation.find_boundaries(labels_edited, connectivity=1, mode='thick', background=0)
    boundaries = boundaries.astype(int)
    img[boundaries>0] = [0,0,255]

    # get unique phenograph clusters
    clusters = list(set(combo['PhenoGraph_clusters'].tolist()))
    
    for cluster in clusters:

        # create list of this cluster's cells
        pos_cells_df = combo[combo['PhenoGraph_clusters'] == cluster]

        for k in range(pos_cells_df.shape[0]):
            img = cv2.putText(img, str(cluster), (int(pos_cells_df['x'].iloc[k]), int(pos_cells_df['y'].iloc[k])), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [255,255,255], 2, cv2.LINE_AA)
            
    # save image
    cv2.imwrite('RGB_clusters_overlaid/' + patient + '_FOV' + str(fov).zfill(2) + '_RGB_cluster.png', img)

20220215_094003_S1_C902_P99_N99_F007_normalized.png
20220215_094003_S1_C902_P99_N99_F007_normalized_cp_masks.tif


In [46]:
# save IDs of RBCs
pos_to_save.to_csv(patient + tp + '_RBCs_FOV07.csv', index=False)

## Concatenate FOVs 1-6 result with FOV 7 result

In [8]:
fovs1_6 = pd.read_csv(patient + tp + '_RBCs_exFOV07.csv')
fov7 = pd.read_csv(patient + tp + '_RBCs_FOV07.csv')
all_fovs = pd.concat([fovs1_6, fov7], ignore_index=True)
all_fovs.to_csv(patient + tp + '_RBCs.csv', index=False)